## gensim으로 네이버 기사 토픽 모델링 해보기

> 토픽 모델링을 적용하기 위해 텍스트를 처리합니다.

> 토픽 모델링 라이브러리인 gensim을 사용해봅니다.




### 토픽 모델링

   *토픽(Topic)은 한국어로는 주제라고 합니다. 토픽 모델링(Topic Modeling)이란 기계 학습 및 자연어 처리 분야에서 토픽이라는 문서 집합의 추상적인 주제를 발견하기 위한 통계적 모델 중 하나로, 텍스트 본문의 숨겨진 의미 구조를 발견하기 위해 사용되는 텍스트 마이닝 기법입니다.
 1. 문서에서 토픽(키워드)을 찾는 과정
 2. 문장을 구성하는 단어 조합으로부터 k개의 단어 묶음을 찾는 과정 
 3. 베이지안 확률 모델이며 , 토픽 모델링의 결과로 각 단어가 각 토픽에 
    속할 확률이 나옴
 4. 잠재 디리클레 할당 (LDA)
    * 각 문서에 여러 개의 토픽이 포함될 수있다 
    * 각 토픽에는 여러개의 단어가 포함 될 수 있다 
    * 문서에 존재하는 모든 단어는 반드시 어떤 토픽에 포함된다 
    * 사람이 글을 쓰는 과정을 생성 모델로 정의한다.
    *토픽 모델링은 문서의 집합에서 토픽을 찾아내는 프로세스를 말합니다. 이는 검색 엔진, 고객 민원 시스템 등과 같이 문서의 주제를 알아내는 일이 중요한 곳에서 사용됩니다. 잠재 디리클레 할당(Latent Dirichlet Allocation, LDA)은 토픽 모델링의 대표적인 알고리즘입니다. 줄여서 LDA라고 합니다.
    
    ** LDA 모델 과정 
       1. 문서들에 사용할 토픽을 고른다 (K개의 토픽)
       2. 토픽 중 하나의 토픽을 고른다 
       3. 그 토픽에 포함된 단어 중에 하나를 고른다 
       4. 단어를 문서에 추가한다 (글을 쓴다)
       5. B번 과정부터 반복한다. 
        
        

In [1]:
# !pip install gensim
# !pip install pickle-mixin

- gensim 설치 
    
자연어를 벡터로 변환하는데 필요한 대부분의 편의기능을 제공하고 있는 라이브러리
Word2vec도 포함(Word2vec : word를 vector로 바꿔주는 테크닉)

### 1. 토픽 모델링을 위한 라이브러리 불러오기

In [2]:
from tqdm import tqdm_notebook # 진행상황을 progress bar 형태로 한눈에 확인할 수 있다
from konlpy.tag import Mecab #Mecab, Okt 등 형태소 분석기 불러오기
import numpy as np
import string # 특수문자
import re
import warnings # 경고 알림 제거
import pickle ## 리스트나 클래스 같은 텍스트가 아닌 자료형을 파일로 저장하기 위해 pickle 모듈 사용함 

import pandas as pd

#gensim 라이브러리 불러오기 

from gensim import corpora
from gensim import models
import matplotlib.pyplot as plt
%matplotlib inline

warnings.filterwarnings("ignore", category=DeprecationWarning) # 경고 알림이 뜨면 모두 무시합니다.

C:\Users\MyCom\anaconda3\envs\text_analysis\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
# pickle 파일 읽기
%time new_df = pd.read_pickle("naver_news_content.pk") #pickle 파일 읽기 
new_df

Wall time: 4 ms


[['   본문 내용     TV플레이어     // TV플레이어     // flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}   \t \t[스포츠경향]  안양대학교 대학일자리센터가 디지털 기초 이해도 확보 및 최신 트렌드에 맞는  IT 역량 강화를 위한 ‘코딩학개론’ 프로그램을 진행한다고  16 일 밝혔다.  10 월  11 일까지 3개월간 진행되는 이번 프로그램은 ‘코드잇’과 연계한 것으로,   IT  실무에서 많이 쓰이는 파이썬 기초, 머신 러닝, 웹퍼블리싱 등으로 교육과정을 구성하고, 학생들이 원하는 과목을 신청하여 자유롭게 수강하도록 개설된 것이 특징이다. 코드잇은 국내에서 유일하게 외주 제작 없이 대기업·아이비리그 출신 개발자들이 콘텐츠를 직접 기획, 제작해 유기적으로 연결된 강의를 제작하는 기업. 최근 글로벌 서비스를 개시해 국내뿐만 아니라 해외에도 높은 수준의 코딩 강의를 제공하고 있으며 높은 고객만족도와 향후 성장 가능성을 인정받아  2021 년 중소벤처기업부의 ‘아기 유니콘 기업’으로 선정되기도 했다.  이번 프로그램은 전공자, 비전공자 관계없이 코딩에 관심 있는 재학생 누구나 신청할 수 있으며 수강률  80 %이상, 기말평가  60 점 이상 달성 시 수료증이 발급된다. 김현태 대학일자리센터 팀장은 “4차 산업시대에 빠르게 변하는 트렌드에 맞춰 전공자들의 실질적인  IT 역량 강화와 비전공자들의 디지털 기초 이해도 확보가 기대된다”고 말했다. 앞서 안양대는 지난해 ‘대학일자리센터’ 개소식을 갖고 폭넓은 진로탐색지원, 맞춤형 진로 선택 지원, 구직활동 지원, 취업경쟁력 강화를 목표로 재학생 뿐만 아니라 지역 청년들의 진로와 취·창업을 위해 다양한 프로그램 운영에 박차를 가하고 있다.    // 본문 내용   ',
  '   본문 내용     TV플레이어     // TV플레이어     // flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback

### 2. 텍스트 전처리 함수 만들기
         -stopwords-ko.txt 텍스트파일 ANSI 형식으로 인코딩 해주기
         - 참고 자료 : https://m.blog.naver.com/tipsware/221727268968

In [4]:
def read_documents(input_file_name):
    """문서들을 주어진 이름의 파일로부터 읽어들여 돌려준다."""
    
    corpus = []
    
    with open(input_file_name, 'rb') as f:
        temp_corpus = pickle.load(f)
        
    for page in temp_corpus:
        corpus += page
    
    return corpus

def text_cleaning(docs):
    # 한국어를 제외한 글자를 제거하는 함수.
    for doc in docs:
        doc = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", doc)

    return docs

def define_stopwords(path):
    
    SW = set()
    # 불용어를 추가하는 방법 1.
    for i in string.punctuation:
        SW.add(i)

    # 불용어를 추가하는 방법 2.
    # stopwords-ko.txt에 직접 추가
    
    with open(path) as f:
        for word in f:
            SW.add(word)

    return SW


def text_tokenizing(corpus, tokenizer):
    
    mecab = Mecab(dicpath='C:\mecab\mecab-ko-dic')
    token_corpus = []
    

    if tokenizer == "noun":
        for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
            token_text = mecab.nouns(corpus[n])
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
                
            token_corpus.append(token_text)
            
    elif tokenized == "morph":
        for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
            token_text = mecab.morphs(corpus[n])
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            token_corpus.append(token_text)

    elif tokenizer == "word":
        for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
            token_text = corpus[n].split()
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            token_corpus.append(token_text)
        

    return token_corpus


input_file_name = "naver_news_content.pk"
documents = read_documents(input_file_name)
SW = define_stopwords("stopwords-ko.txt")
cleaned_text = text_cleaning(documents)
tokenized_text = text_tokenizing(cleaned_text, tokenizer="noun") #tokenizer= "noun" or "word"

Preprocessing:   0%|          | 0/3 [00:00<?, ?it/s]

문서 읽기의 과정은 앞서 단어 임베딩의 경우와 다르지 않다. 다음 과정은 문서-단어 행렬을 만드는 과정이다.

In [17]:
print(tokenized_text[0])

['본문', '내용', '플레이어', '플레이어', '오류', '우회', '함수', '추가', '스포츠', '경향', '안양', '대학교', '대학', '일자리', '센터', '디지털', '기초', '이해', '확보', '최신', '트렌드', '역량', '강화', '코딩', '학개', '프로그램', '진행', '개월', '진행', '이번', '프로그램', '코드', '연계', '실무', '파이썬', '기초', '머신', '러닝', '퍼블리싱', '교육', '과정', '구성', '학생', '과목', '신청', '자유', '수강', '개설', '특징', '코드', '국내', '유일', '외주', '제작', '기업', '아이비리그', '출신', '개발자', '콘텐츠', '기획', '제작', '유기', '연결', '강의', '제작', '기업', '최근', '글로벌', '서비스', '개시', '국내', '해외', '수준', '코딩', '강의', '제공', '고객', '만족도', '향후', '성장', '가능', '중소', '벤처', '기업', '아기', '유니콘', '기업', '선정', '이번', '프로그램', '전공', '전공', '코딩', '관심', '재학', '누구', '신청', '수강', '이상', '기말', '평가', '이상', '달성', '수료증', '발급', '김현태', '대학', '일자리', '센터', '팀장', '산업', '시대', '트렌드', '전공', '실질', '역량', '강화', '전공', '디지털', '기초', '이해', '확보', '기대', '양대', '지난해', '대학', '일자리', '센터', '개소식', '진로', '탐색', '지원', '맞춤', '진로', '선택', '지원', '구직', '활동', '지원', '취업', '경쟁력', '강화', '목표', '재학', '지역', '청년', '진로', '창업', '프로그램', '운영', '박차', '본문', '내용']


### 3. 토픽 모델링에 사용할 함수들 확인하기

In [6]:
# 문서-단어 행렬 만들기
# 어휘(vocabulary) 학습
#from gensim import corpora
dictionary = corpora.Dictionary(tokenized_text)
# 문서-단어 행렬(document-term matrix) 생성
corpus = [dictionary.doc2bow(text) for text in tokenized_text]

In [7]:
print(dictionary)

Dictionary(171 unique tokens: ['가능', '강의', '강화', '개발자', '개설']...)


In [8]:
corpus[0][:5]

[(0, 1), (1, 2), (2, 3), (3, 1), (4, 1)]

### 4.TF-IDF(단어 빈도-역 문서 빈도, Term Frequency-Inverse Document Frequency)

  TF-IDF(Term Frequency-Inverse Document Frequency)는 단어의 빈도와 역 문서 빈도(문서의 빈도에 특정 식을 취함)를 사용하여 DTM 내의 각 단어들마다 중요한 정도를 가중치로 주는 방법입니다. 사용 방법은 우선 DTM을 만든 후, TF-IDF 가중치를 부여합니다.

TF-IDF는 주로 문서의 유사도를 구하는 작업, 검색 시스템에서 검색 결과의 중요도를 정하는 작업, 문서 내에서 특정 단어의 중요도를 구하는 작업 등에 쓰일 수 있습니다.

TF-IDF는 TF와 IDF를 곱한 값을 의미하는데 이를 식으로 표현해보겠습니다. 문서를 d, 단어를 t, 문서의 총 개수를 n이라고 표현할 때 TF, DF, IDF는 각각 다음과 같이 정의할 수 있습니다.

(1) tf(d,t) : 특정 문서 d에서의 특정 단어 t의 등장 횟수.
생소한 글자때문에 어려워보일 수 있지만, 잘 생각해보면 TF는 이미 앞에서 구한 적이 있습니다. TF는 앞에서 배운 DTM의 예제에서 각 단어들이 가진 값들입니다. DTM이 각 문서에서의 각 단어의 등장 빈도를 나타내는 값이었기 때문입니다.

(2) df(t) : 특정 단어 t가 등장한 문서의 수.
여기서 특정 단어가 각 문서, 또는 문서들에서 몇 번 등장했는지는 관심가지지 않으며 오직 특정 단어 t가 등장한 문서의 수에만 관심을 가집니다. 앞서 배운 DTM에서 바나나는 문서2와 문서3에서 등장했습니다. 이 경우, 바나나의 df는 2입니다. 문서3에서 바나나가 두 번 등장했지만, 그것은 중요한 게 아닙니다. 심지어 바나나란 단어가 문서2에서 100번 등장했고, 문서3에서 200번 등장했다고 하더라도 바나나의 df는 2가 됩니다.

(3) idf(d, t) : df(t)에 반비례하는 수.


In [9]:
# TFIDF 문서-단어 행렬 생성
# TF-IDF로 corpus를 변환
# corpus(말뭉치) = > nltk 패키지의 서브패키지에서는 다앙한 연구용 말뭉치를 제공한다 
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
corpus_tfidf[0][:5]

[(0, 0.06084982941388938),
 (1, 0.12169965882777876),
 (2, 0.18254948824166817),
 (3, 0.06084982941388938),
 (5, 0.06084982941388938)]

In [21]:
# LDA모델 생성
model = models.ldamodel.LdaModel(corpus, num_topics=4, id2word=dictionary)

In [11]:
model.show_topic(3, 10)

[('과정', 0.036929026),
 ('안랩', 0.03349875),
 ('교육', 0.032891143),
 ('강사', 0.019402198),
 ('지원', 0.018727802),
 ('디지털', 0.018118385),
 ('프로그램', 0.015849968),
 ('경력', 0.012340712),
 ('수료생', 0.012300622),
 ('파이썬', 0.0103326235)]

### 4. 토픽 모델링을 추가하여 코드 완성하기

   - LDA 모델의 주요 입력값은 dictionary와 corpus이다
   - 빈도수 기반으로 BoW의 행렬 또는 TF-IDF 행렬로 corpus를 나타냄

In [20]:
# 토픽 개수, 키워드 개수를 정해주는 변수를 추가.
NUM_TOPICS = 3

NUM_TOPIC_WORDS = 30


def build_doc_term_mat(documents):
    # 문서-단어 행렬 만들어주는 함수.
    print("Building document-term matrix.")
    dictionary = corpora.Dictionary(documents) #dictionary를 이용하여  corpus 생성함
    corpus = [dictionary.doc2bow(document) for document in documents]
        
    return corpus, dictionary


def print_topic_words(model):

    # 토픽 모델링 결과를 출력해 주는 함수.
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

# document-term matrix를 만들고,
corpus, dictionary = build_doc_term_mat(tokenized_text)
# LDA를 실행.
model = models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
# 결과를 출력.
print_topic_words(model)

Building document-term matrix.

Printing topic words.

Topic ID: 0
	교육	0.020134063437581062
	과정	0.01933959499001503
	지원	0.01856546476483345
	프로그램	0.01719101145863533
	전공	0.016903992742300034
	강화	0.015583008527755737
	센터	0.01377133745700121
	코딩	0.013474454171955585
	일자리	0.013459446839988232
	디지털	0.013236879371106625
	기업	0.013011089526116848
	대학	0.012913161888718605
	기초	0.012707662768661976
	내용	0.011979364790022373
	진로	0.011881052516400814
	제작	0.011384266428649426
	플레이어	0.011376134119927883
	본문	0.011355482041835785
	안랩	0.010759378783404827
	이번	0.010608116164803505
	코드	0.010289400815963745
	신청	0.01004485972225666
	국내	0.009928843937814236
	역량	0.009752754122018814
	재학	0.009469701908528805
	강의	0.009315531700849533
	확보	0.009268994443118572
	수강	0.009232008829712868
	이해	0.00896440725773573
	이상	0.008546862751245499


Topic ID: 1
	안랩	0.03611671179533005
	교육	0.035984765738248825
	과정	0.028996383771300316
	프로그램	0.019850241020321846
	디지털	0.01963992789387703
	강사	0.01785849593579769
	지원	0.0162763241678

### 5. pyLDAvis를 통한 토픽 모델링 결과 시각화하기

     -LDA 시각화를 위해서는 pyLDAvis의 설치가 필요합니다. 윈도우의 명령 프롬프트나 MAC/UNIX의 터미널에서 아래의 명령을 수행하여 pyLDAvis를 설치하시기 바랍니다.



In [18]:
!pip install pyLDAvis

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for pyLDAvis: filename=pyLDAvis-3.2.2-py2.py3-none-any.whl size=135593 sha256=262e5ce09089c4b45730aaa1d68d2998f5fc6e1fabfb7326d9e778caa1cb1e74
  Sto

In [19]:
# pyLDAvis 불러오기
import pyLDAvis
import pyLDAvis.gensim

# pyLDAvis를 jupyter notebook에서 실행할 수 있게 활성화.
pyLDAvis.enable_notebook()

# pyLDAvis 실행.
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data

# pyLDAvis 실행결과 깃허브에서 안보여집니다.
# 밑에 링크를 통해 실행결과 확인 가능합니다
#https://nbviewer.jupyter.org/github/SEONGJAE-YOO/MachineLearning-dataAnalysis/blob/main/KakaoTalk%20conversation%20analysis%20using%20Text%20Mining/%EA%B8%B0%EC%82%AC%20%ED%85%8D%EC%8A%A4%ED%8A%B8%20%EC%A0%95%EC%A0%9C%20-%20gensim%EC%9D%84%20%EC%9D%B4%EC%9A%A9%ED%95%9C%20%ED%86%A0%ED%94%BD%EB%AA%A8%EB%8D%B8%EB%A7%81%20%EB%B6%84%EC%84%9D.ipynb


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.015054  0.000079       1        1  67.469459
0     -0.006988 -0.001702       2        1  27.501048
2     -0.008066  0.001623       3        1   5.029493, topic_info=     Term      Freq     Total Category  logprob  loglift
14     교육  8.000000  8.000000  Default  30.0000  30.0000
142    안랩  6.000000  6.000000  Default  29.0000  29.0000
12     과정  7.000000  7.000000  Default  28.0000  28.0000
30    디지털  6.000000  6.000000  Default  27.0000  27.0000
100  프로그램  8.000000  8.000000  Default  26.0000  26.0000
..    ...       ...       ...      ...      ...      ...
74     전공  0.207155  6.884364   Topic3  -4.6848  -0.5137
46     수강  0.173327  4.043903   Topic3  -4.8630  -0.1599
28     대학  0.191168  6.377486   Topic3  -4.7651  -0.5175
94    트렌드  0.163609  3.885189   Topic3  -4.9207  -0.1776
17     국내  0.160533  4.148696   Topic3  -4.9397  -0.2622

[190 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
112       1  0.786353  가상현실
113       1  0.530857    강사
113       2  0.530857    강사
1         1  0.748179    강의
1         2  0.249393    강의
...     ...       ...   ...
107       2  0.242000    확보
168       1  0.842602    활약
169       1  0.778126    후속
169       2  0.778126    후속
170       1  0.841711    희망

[158 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3])